In [27]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *
from tqdm import tqdm
import pandas as pd
import numpy as np
import rdchiral
from tqdm import tqdm
from common import *
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdChemReactions
from rdkit.Chem.Draw import rdMolDraw2D
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import SVG, display
from rxnmapper import RXNMapper
from rdkit.Chem import AllChem
from rdchiral.main import rdchiralRun, rdchiralRunText, rdchiralReaction, rdchiralReactants
from rdchiral.template_extractor import mols_from_smiles_list, replace_deuterated, clear_mapnum, \
    get_tagged_atoms_from_mols, \
    get_tagged_atoms_from_mol, atoms_are_different, find_map_num, get_tetrahedral_atoms, set_isotope_to_equal_mapnum, \
    get_frag_around_tetrahedral_center, check_tetrahedral_centers_equivalent, clear_isotope, get_changed_atoms, \
    get_special_groups, expand_atoms_to_use, expand_atoms_to_use_atom, convert_atom_to_wildcard, reassign_atom_mapping, \
    get_strict_smarts_for_atom, expand_changed_atom_tags, get_fragments_for_changed_atoms, canonicalize_transform, \
    canonicalize_template, bond_to_label

In [34]:
mnxreac_smile_atom_mapping_rules_file_path = '../../../Data/rules/MNXreaction_smiles_atommap.csv'
mnxreac_file_path = '../../../Data/database/MNX_reac_prop.tsv'
mnxreac_smile_file_path = '../../../Data/database/MNXreaction_smile.csv'
total_met_inchikey0_file = '../../../Data/total_met_inchikey0.pickle'


In [28]:
def extract_from_reaction(reaction):
    reactants = mols_from_smiles_list(replace_deuterated(reaction['reactants']).split('.'))
    products = mols_from_smiles_list(replace_deuterated(reaction['products']).split('.'))

    # if rdkit cant understand molecule, return
    if None in reactants: return {'reaction_id': reaction['_id']}
    if None in products: return {'reaction_id': reaction['_id']}

    # try to sanitize molecules
    try:
        for i in range(len(reactants)):
            reactants[i] = AllChem.RemoveHs(reactants[i])  # *might* not be safe
        for i in range(len(products)):
            products[i] = AllChem.RemoveHs(products[i])  # *might* not be safe
        [Chem.SanitizeMol(mol) for mol in reactants + products]  # redundant w/ RemoveHs
        [mol.UpdatePropertyCache() for mol in reactants + products]
    except Exception as e:
        # can't sanitize -> skip
        print(e)
        print('Could not load SMILES or sanitize')
        print('ID: {}'.format(reaction['_id']))
        return {'reaction_id': reaction['_id']}

    are_unmapped_product_atoms = False
    extra_reactant_fragment = ''
    for product in products:
        prod_atoms = product.GetAtoms()
        if sum([a.HasProp('molAtomMapNumber') for a in prod_atoms]) < len(prod_atoms):
            if VERBOSE: print('Not all product atoms have atom mapping')
            if VERBOSE: print('ID: {}'.format(reaction['_id']))
            are_unmapped_product_atoms = True

    if are_unmapped_product_atoms:  # add fragment to template
        for product in products:
            prod_atoms = product.GetAtoms()
            # Get unmapped atoms
            unmapped_ids = [
                a.GetIdx() for a in prod_atoms if not a.HasProp('molAtomMapNumber')
            ]
            if len(unmapped_ids) > MAXIMUM_NUMBER_UNMAPPED_PRODUCT_ATOMS:
                # Skip this example - too many unmapped product atoms!
                return
            # Define new atom symbols for fragment with atom maps, generalizing fully
            atom_symbols = ['[{}]'.format(a.GetSymbol()) for a in prod_atoms]
            # And bond symbols...
            bond_symbols = ['~' for b in product.GetBonds()]
            if unmapped_ids:
                extra_reactant_fragment += AllChem.MolFragmentToSmiles(
                    product, unmapped_ids,
                    allHsExplicit=False, isomericSmiles=USE_STEREOCHEMISTRY,
                    atomSymbols=atom_symbols, bondSymbols=bond_symbols
                ) + '.'
        if extra_reactant_fragment:
            extra_reactant_fragment = extra_reactant_fragment[:-1]
            if VERBOSE: print('    extra reactant fragment: {}'.format(extra_reactant_fragment))

        # Consolidate repeated fragments (stoichometry)
        extra_reactant_fragment = '.'.join(sorted(list(set(extra_reactant_fragment.split('.')))))

    if None in reactants + products:
        print('Could not parse all molecules in reaction, skipping')
        print('ID: {}'.format(reaction['_id']))
        return {'reaction_id': reaction['_id']}

    # Calculate changed atoms
    changed_atoms, changed_atom_tags, err = get_changed_atoms(reactants, products)
    if err:
        if VERBOSE:
            print('Could not get changed atoms')
            print('ID: {}'.format(reaction['_id']))
        return
    if not changed_atom_tags:
        if VERBOSE:
            print('No atoms changed?')
            print('ID: {}'.format(reaction['_id']))
        # print('Reaction SMILES: {}'.format(example_doc['RXN_SMILES']))
        return {'reaction_id': reaction['_id']}

    try:
        # Get fragments for reactants
        reactant_fragments, intra_only, dimer_only = get_fragments_for_changed_atoms(reactants, changed_atom_tags,
                                                                                     radius=1, expansion=[],
                                                                                     category='reactants')
        # Get fragments for products
        # (WITHOUT matching groups but WITH the addition of reactant fragments)
        product_fragments, _, _ = get_fragments_for_changed_atoms(products, changed_atom_tags,
                                                                  radius=0,
                                                                  expansion=expand_changed_atom_tags(changed_atom_tags,
                                                                                                     reactant_fragments),
                                                                  category='products')
        # while loop for the radius definition
        r = 1  # radius
        while len(product_fragments.split('.')) > len(products) or len(reactant_fragments.split('.')) > len(reactants):
            r += 1
            reactant_fragments, intra_only, dimer_only = get_fragments_for_changed_atoms(reactants, changed_atom_tags,
                                                                                         radius=r, expansion=[],
                                                                                         category='reactants')
            product_fragments, _, _ = get_fragments_for_changed_atoms(products, changed_atom_tags,
                                                                      radius=0,
                                                                      expansion=expand_changed_atom_tags(
                                                                          changed_atom_tags,
                                                                          reactant_fragments),
                                                                      category='products')
            if r > 10:
                break

    except ValueError as e:
        if VERBOSE:
            print(e)
            print(reaction['_id'])
        return {'reaction_id': reaction['_id']}

    # Put together and canonicalize (as best as possible)
    rxn_string = '{}>>{}'.format(reactant_fragments, product_fragments)
    rxn_canonical = canonicalize_transform(rxn_string)
    # Change from inter-molecular to intra-molecular
    rxn_canonical_split = rxn_canonical.split('>>')
    rxn_canonical = rxn_canonical_split[0][1:-1].replace(').(', '.') + \
                    '>>' + rxn_canonical_split[1][1:-1].replace(').(', '.')

    reactants_string = rxn_canonical.split('>>')[0]
    products_string = rxn_canonical.split('>>')[1]

    retro_canonical = products_string + '>>' + reactants_string  # not used in this template

    # Load into RDKit
    rxn = AllChem.ReactionFromSmarts(retro_canonical)
    if rxn.Validate()[1] != 0:
        print('Could not validate reaction successfully')
        print('ID: {}'.format(reaction['_id']))
        print('retro_canonical: {}'.format(retro_canonical))
        if VERBOSE: raw_input('Pausing...')
        return {'reaction_id': reaction['_id']}

    template = {
        'productSMARTs': products_string,
        'reactantSMARTs': reactants_string,
        'products': reaction['products'],
        'reactants': reaction['reactants'],
        'reaction_smarts': rxn_canonical,
        'retro_reaction_smarts':retro_canonical,
        'intra_only': intra_only,
        'dimer_only': dimer_only,
        'reaction_id': reaction['_id'],
        'necessary_reagent': extra_reactant_fragment,
        'radius': r,
    }

    return template

In [29]:
def extract_MNXreaction_rules(mnxreac_smile_atom_mapping,set_confidence_score):
    failed_extract = []
    for index,row in tqdm(mnxreac_smile_atom_mapping.iterrows(), total=len(mnxreac_smile_atom_mapping)):
        rxn_id = row['MNX_ID']
        atommap = row['RxnMapped']
        confidencescore = row['confidence']
        if confidencescore > set_confidence_score:
            reaction = {
            'reactants': atommap.split('>')[0],
            'products': atommap.split('>')[-1],
            '_id': rxn_id,
            }
            try:

                template = extract_from_reaction(reaction)
                mnxreac_smile_atom_mapping.loc[index,'ReactantsSMARTs'] = template['reactantSMARTs']
                mnxreac_smile_atom_mapping.loc[index,'ProductSMARTs'] = template['productSMARTs']
                mnxreac_smile_atom_mapping.loc[index,'RetroRules'] = template['retro_reaction_smarts']
                mnxreac_smile_atom_mapping.loc[index,'Radius'] = template['radius']

            except:
                # print('failed extract:', row['MNX_ID'])
                failed_extract.append(row['MNX_ID'])
    print('done')
    print('failed extract:',failed_extract)

    mnxreac_smile_atom_mapping.dropna(subset=['RetroRules'], inplace=True)
    mnxreac_smile_atom_mapping.reset_index(drop=True, inplace=True)
    return mnxreac_smile_atom_mapping
    # mnxreac_smile_atom_mapping.to_csv(mnxreac_smile_atom_mapping_rules_file_path,index=None)

In [9]:
retrorule = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
retrorule.head()

,MNX_ID,equation,REFERENCE,classifs,is_balanced,is_transport,substrate_smiles,product_smiles,equ_name,equ_smiles,deprecated_equ,deprecated_equ_smiles,RxnMapped,confidence
0,MNXR100024,1 MNXM1@MNXD1 + 1 MNXM37@MNXD1 + 1 MNXM40333@M...,rheaR:16169,6.3.1.2,B,NaN,NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H],[NH4+].[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-],L-glutamine + H2O <=> NH4(+) + L-glutamate,NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H]>>[NH4+]...,H(+) + ADP + phosphate <=> ATP + H2O,[H+].Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])...,[NH3+:1][C@@H:2]([CH2:3][CH2:4][C:5](=[O:6])[N...,0.984627
1,MNXR100024_reverse,1 MNXM3@MNXD1 + 1 MNXM729302@MNXD1 + 1 MNXM741...,rheaR:16169,6.3.1.2,B,NaN,[NH4+].[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-],NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H],NH4(+) + L-glutamate <=> L-glutamine + H2O,[NH4+].[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-]>>NC(...,ATP + H2O <=> H(+) + ADP + phosphate,Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])OP(=O...,[C:2](=[O:3])([CH2:4][CH2:5][C@H:6]([NH3+:7])[...,0.876969
2,MNXR100030,1 MNXM37@MNXD1 + 1 WATER@MNXD1 <=> 1 MNXM72930...,rheaR:15889,1.4.1.13;1.4.1.14;1.4.7.1;2.4.2;2.4.2.14;3.5.1...,B,NaN,NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H],[NH4+].[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-],L-glutamine + H2O <=> NH4(+) + L-glutamate,NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H]>>[NH4+]...,<=>,>>,[NH3+:1][C@@H:2]([CH2:3][CH2:4][C:5](=[O:6])[N...,0.984627
3,MNXR100030_reverse,1 MNXM729302@MNXD1 + 1 MNXM741173@MNXD1 <=> 1 ...,rheaR:15889,1.4.1.13;1.4.1.14;1.4.7.1;2.4.2;2.4.2.14;3.5.1...,B,NaN,[NH4+].[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-],NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H],NH4(+) + L-glutamate <=> L-glutamine + H2O,[NH4+].[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-]>>NC(...,<=>,>>,[C:2](=[O:3])([CH2:4][CH2:5][C@H:6]([NH3+:7])[...,0.876969
4,MNXR100060,1 MNXM10@MNXD1 + 2 MNXM1@MNXD1 + 1 MNXM222@MNX...,rheaR:20001,1.2.1;1.2.1.19;1.2.1.21;1.2.1.3;1.2.1.4;1.2.1....,B,NaN,O=C([O-])CO,O=CCO.[H]O[H],glycolate <=> glycolaldehyde + H2O,O=C([O-])CO>>O=CCO.[H]O[H],NADH + 2.0 H(+) <=> NAD(+),NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O)(...,[O-:1][C:3](=[O:2])[CH2:4][OH:5]>>[OH2:1].[O:2...,0.813147


In [10]:
retrorule[retrorule['MNX_ID']=='MNXR151346']

,MNX_ID,equation,REFERENCE,classifs,is_balanced,is_transport,substrate_smiles,product_smiles,equ_name,equ_smiles,deprecated_equ,deprecated_equ_smiles,RxnMapped,confidence


In [23]:
retrorule_MNXR188249 = retrorule[retrorule['MNX_ID']=='MNXR188249']


In [30]:
retrorule_MNXR188249 = extract_MNXreaction_rules(retrorule_MNXR188249,0.1)

  0%|          | 0/1 [00:00<?, ?it/s]/tmp/ipykernel_3440666/267956634.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mnxreac_smile_atom_mapping.loc[index,'ReactantsSMARTs'] = template['reactantSMARTs']
/tmp/ipykernel_3440666/267956634.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mnxreac_smile_atom_mapping.loc[index,'ProductSMARTs'] = template['productSMARTs']
/tmp/ipykernel_3440666/267956634.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

done
failed extract: []



/tmp/ipykernel_3440666/267956634.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mnxreac_smile_atom_mapping.dropna(subset=['RetroRules'], inplace=True)


In [31]:
retrorule_MNXR188249

,MNX_ID,equation,REFERENCE,classifs,is_balanced,is_transport,substrate_smiles,product_smiles,equ_name,equ_smiles,deprecated_equ,deprecated_equ_smiles,RxnMapped,confidence,ReactantsSMARTs,ProductSMARTs,RetroRules,Radius
0,MNXR188249,1 MNXM1103365@MNXD1 + 1 MNXM1422@MNXD1 + 1 MNX...,rheaR:24793,2.4.2;2.4.2.-;4.1.3.-;4.3.1;4.3.2.10;4.3.2.M2,B,NaN,NC(=O)c1ncn([C@@H]2O[C@H](COP(=O)([O-])[O-])[C...,NC(=O)c1ncn([C@@H]2O[C@H](COP(=O)([O-])[O-])[C...,5-amino-1-(5-phospho-beta-D-ribosyl)imidazole-...,NC(=O)c1ncn([C@@H]2O[C@H](COP(=O)([O-])[O-])[C...,H(+) <=>,[H+]>>,[NH2:11][C:12](=[O:13])[c:14]1[n:15][cH:16][n:...,0.132592,[#7;a:1]:[c:2]:[c:3](:[#7;a:4])-[NH2;D1;+0:5]....,[#7;a:1]:[c:2]:[c:3](:[#7;a:4])-[NH;D2;+0:5]-[...,[#7;a:1]:[c:2]:[c:3](:[#7;a:4])-[NH;D2;+0:5]-[...,1.0


In [33]:
retrorule_MNXR188249['equ_smiles'][0]

'NC(=O)c1ncn([C@@H]2O[C@H](COP(=O)([O-])[O-])[C@@H](O)[C@H]2O)c1N.O=P([O-])([O-])OC[C@@H](O)[C@@H](O)c1c[nH]cn1.[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-]>>NC(=O)c1ncn([C@@H]2O[C@H](COP(=O)([O-])[O-])[C@@H](O)[C@H]2O)c1NC=NCC(=O)[C@H](O)[C@H](O)COP(=O)([O-])[O-].NC(=O)CC[C@H]([NH3+])C(=O)[O-]'

In [36]:
total_met_inchikey0 = load_pickle(total_met_inchikey0_file)
reactant_smiles = retrorule_MNXR188249['equ_smiles'][0].split('>>')[0]
product_smiles = retrorule_MNXR188249['equ_smiles'][0].split('>>')[1]
for smi in reactant_smiles.split('.'):
    if smiles2inchikey0(smi) not in total_met_inchikey0:
        print(smi)
for smi in product_smiles.split('.'):
    if smiles2inchikey0(smi) not in total_met_inchikey0:
        print(smi)

In [32]:
retrorule_tmp = retrorule_MNXR188249['RetroRules'][0].split('>>')
retrorule_tmp = '(' + retrorule_tmp[0] + ')>>' + '(' + retrorule_tmp[1] + ')'            
product_smile_tmp = retrorule_MNXR188249['product_smiles'][0]
    # print('product_smile_tmp',product_smile_tmp)
product_mol = Chem.MolFromSmiles(product_smile_tmp)
reaction = AllChem.ReactionFromSmarts(retrorule_tmp)
reactant_smiles = reaction.RunReactants((product_mol,))
smiles = [Chem.MolToSmiles(x[0]) for x in reactant_smiles]
reactant_smiles = list(set(smiles))
reactant_smiles

['NC(=O)c1ncn([C@@H]2O[C@H](COP(=O)([O-])[O-])[C@@H](O)[C@H]2O)c1N.O=P([O-])([O-])OC[C@@H](O)[C@@H](O)c1c[nH]cn1.[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-]']

In [15]:
mnxreac = pd.read_csv(mnxreac_file_path, sep='\t')
mnxreac.head()

,ID,mnx_equation,reference,classifs,is_balanced,is_transport
0,MNXR01,1 MNXM01@MNXD1 = 1 MNXM1@MNXD1,mnx:MNXR01,NaN,B,NaN
1,MNXR02,1 MNXM1@MNXD1 = 1 MNXM1@MNXD2,mnx:MNXR02,NaN,B,T
2,MNXR03,1 MNXM01@MNXD1 = 1 MNXM01@MNXD2,mnx:MNXR03,NaN,B,T
3,MNXR100000,1 MNXM10958@MNXD1 + 1 MNXM1104529@MNXD1 = 1 MN...,biggR:GALNACT5g,NaN,NaN,NaN
4,MNXR100001,1 MNXM1100890@MNXD1 + 1 MNXM147296@MNXD1 = 1 M...,biggR:GALNTg,NaN,NaN,NaN


In [16]:
mnxreac[mnxreac['ID']=='MNXR151346']


,ID,mnx_equation,reference,classifs,is_balanced,is_transport
32225,MNXR151346,1 MNXM13@MNXD1 + 1 MNXM727664@MNXD1 = 1 MNXM1@...,rheaR:54368,NaN,B,NaN


In [18]:
mnxreac_smile = pd.read_csv(mnxreac_smile_file_path)
mnxreac_smile.head()

,MNX_ID,equation,REFERENCE,classifs,is_balanced,is_transport,substrate_smiles,product_smiles,equ_name,equ_smiles,deprecated_equ,deprecated_equ_smiles
0,MNXR100024,1 MNXM1@MNXD1 + 1 MNXM37@MNXD1 + 1 MNXM40333@M...,rheaR:16169,6.3.1.2,B,NaN,NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H],[NH4+].[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-],L-glutamine + H2O <=> NH4(+) + L-glutamate,NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H]>>[NH4+]...,H(+) + ADP + phosphate <=> ATP + H2O,[H+].Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])...
1,MNXR100030,1 MNXM37@MNXD1 + 1 WATER@MNXD1 <=> 1 MNXM72930...,rheaR:15889,1.4.1.13;1.4.1.14;1.4.7.1;2.4.2;2.4.2.14;3.5.1...,B,NaN,NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H],[NH4+].[NH3+][C@@H](CCC(=O)[O-])C(=O)[O-],L-glutamine + H2O <=> NH4(+) + L-glutamate,NC(=O)CC[C@H]([NH3+])C(=O)[O-].[H]O[H]>>[NH4+]...,<=>,>>
2,MNXR100060,1 MNXM10@MNXD1 + 2 MNXM1@MNXD1 + 1 MNXM222@MNX...,rheaR:20001,1.2.1;1.2.1.19;1.2.1.21;1.2.1.3;1.2.1.4;1.2.1....,B,NaN,O=C([O-])CO,O=CCO.[H]O[H],glycolate <=> glycolaldehyde + H2O,O=C([O-])CO>>O=CCO.[H]O[H],NADH + 2.0 H(+) <=> NAD(+),NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O)(...
3,MNXR100063,1 MNXM1048@MNXD1 <=> 1 MNXM736699@MNXD1,rheaR:32239,5.5.1.19,B,NaN,CC(C)=CCC/C(C)=C/C=C/C(C)=C/C=C/C(C)=C/C=C/C=C...,CC1=C(/C=C/C(C)=C/C=C/C(C)=C/C=C/C=C(C)/C=C/C=...,gamma-carotene <=> all-trans-beta-carotene,CC(C)=CCC/C(C)=C/C=C/C(C)=C/C=C/C(C)=C/C=C/C=C...,<=>,>>
4,MNXR100064,1 MNXM1102150@MNXD1 + 1 MNXM1277@MNXD1 + 1 MNX...,seedR:rxn09498,2.1.2.10,B,NaN,Nc1nc2c(c(=O)[nH]1)N1CN(c3ccc(C(=O)N[C@@H](CCC...,Nc1nc2c(c(=O)[nH]1)N[C@@H](CNc1ccc(C(=O)N[C@@H...,"(6R)-5,10-methylene-5,6,7,8-tetrahydrofolate +...",Nc1nc2c(c(=O)[nH]1)N1CN(c3ccc(C(=O)N[C@@H](CCC...,<=> H(+),>>[H+]


In [19]:
mnxreac_smile[mnxreac_smile['MNX_ID']=='MNXR151346']


,MNX_ID,equation,REFERENCE,classifs,is_balanced,is_transport,substrate_smiles,product_smiles,equ_name,equ_smiles,deprecated_equ,deprecated_equ_smiles
12439,MNXR151346,1 MNXM13@MNXD1 + 1 MNXM727664@MNXD1 <=> 1 MNXM...,rheaR:54368,NaN,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN
